# pydantic

### 1. Validation decorator: The `validate_arguments` decorator allows the arguments passed to a function to be parsed and validated using the function's annotations before the function is called.

In [1]:
from pydantic import validate_arguments, ValidationError


@validate_arguments
def repeat( s: str, count: int, *, seperator: bytes = b'' )-> str:
    b = s.encode()
    return seperator.join( b for _ in range( count ))


In [2]:
a = repeat('hello', 3)
print(a)

b'hellohellohello'


In [3]:
b = repeat( 'hello', 'wrong')

ValidationError: 1 validation error for Repeat
count
  value is not a valid integer (type=type_error.integer)

In [4]:
try:
    c = repeat('hello', 'wrong')
except ValidationError as exc:
    print(exc)

1 validation error for Repeat
count
  value is not a valid integer (type=type_error.integer)


## Argument Types

In [6]:
from pathlib import Path
from typing import Pattern, Optional

from pydantic import validate_arguments, DirectoryPath

@validate_arguments
def find_file( path: DirectoryPath, regex: Pattern, max=None) -> Optional[Path]:
    for i, f in enumerate( path.glob('**/*')):
        if max and i > max:
            return 
        if f.is_file() and regex.fullmatch( str(f.relative_to(path) ) ):
            return f      
    

In [8]:
print( find_file(r'C:\Users\hl3\OneDrive - DXC Production\heap\py\work_space', '^file.*'))

C:\Users\hl3\OneDrive - DXC Production\heap\py\work_space\file.txt


In [9]:
print( find_file(r'C:\Users\hl3\OneDrive - DXC Production\heap\py\work_space', '^file.*', max=3 ))

None


## Function Signatures<br>
- positional or keyword arguments with or without defaults
- variable positional arguments defined via * (often *args)
- variable keyword arguments defined via ** (often **kwargs)
- keyword only arguments - arguments after *,
- positional only arguments - arguments before , / (new in python 3.8)

In [10]:
from pydantic import validate_arguments


@validate_arguments
def pos_or_kw(a: int, b: int = 2) -> str:
    return f'a={a} b={b}'


print(pos_or_kw(1))
#> a=1 b=2
print(pos_or_kw(a=1))
#> a=1 b=2
print(pos_or_kw(1, 3))
#> a=1 b=3
print(pos_or_kw(a=1, b=3))
#> a=1 b=3

a=1 b=2
a=1 b=2
a=1 b=3
a=1 b=3


In [11]:
@validate_arguments
def kw_only(*, a: int, b: int = 2) -> str:
    return f'a={a} b={b}'


print(kw_only(a=1))
#> a=1 b=2
print(kw_only(a=1, b=3))
#> a=1 b=3

a=1 b=2
a=1 b=3


In [12]:
@validate_arguments
def pos_only(a: int, b: int = 2, /) -> str:  # python 3.8 only
    return f'a={a} b={b}'


print(pos_only(1))
#> a=1 b=2
print(pos_only(1, 2))
#> a=1 b=2

a=1 b=2
a=1 b=2


In [13]:
@validate_arguments
def var_args(*args: int) -> str:
    return str(args)


print(var_args(1))
#> (1,)
print(var_args(1, 2))
#> (1, 2)
print(var_args(1, 2, 3))
#> (1, 2, 3)

(1,)
(1, 2)
(1, 2, 3)


In [14]:
@validate_arguments
def var_kwargs(**kwargs: int) -> str:
    return str(kwargs)


print(var_kwargs(a=1))
#> {'a': 1}
print(var_kwargs(a=1, b=2))
#> {'a': 1, 'b': 2}

{'a': 1}
{'a': 1, 'b': 2}


In [15]:
@validate_arguments
def armageddon(
    a: int,
    /,  # python 3.8 only
    b: int,
    c: int = None,
    *d: int,
    e: int,
    f: int = None,
    **g: int,
) -> str:
    return f'a={a} b={b} c={c} d={d} e={e} f={f} g={g}'


print(armageddon(1, 2, e=3))
#> a=1 b=2 c=None d=() e=3 f=None g={}
print(armageddon(1, 2, 3, 4, 5, 6, e=8, f=9, g=10, spam=11))
#> a=1 b=2 c=3 d=(4, 5, 6) e=8 f=9 g={'g': 10, 'spam': 11}

a=1 b=2 c=None d=() e=3 f=None g={}
a=1 b=2 c=3 d=(4, 5, 6) e=8 f=9 g={'g': 10, 'spam': 11}


## Using Field to describe function arguments<br>
Field can also be used with validate_arguments to provide extra information about the field and validations.

In [16]:
from datetime import datetime
from pydantic import validate_arguments, Field, ValidationError
from pydantic.typing import Annotated


@validate_arguments
def how_many(num: Annotated[int, Field(gt=10)]):
    return num


try:
    how_many(1)
except ValidationError as e:
    print(e)
    """
    1 validation error for HowMany
    num
      ensure this value is greater than 10 (type=value_error.number.not_gt;
    limit_value=10)
    """


@validate_arguments
def when(dt: datetime = Field(default_factory=datetime.now)):
    return dt


print(type(when()))

1 validation error for HowMany
num
  ensure this value is greater than 10 (type=value_error.number.not_gt; limit_value=10)
<class 'datetime.datetime'>


## Async Functions

In [17]:
import asyncio
from pydantic import PositiveInt, ValidationError, validate_arguments


@validate_arguments
async def get_user_email(user_id: PositiveInt):
    # `conn` is some fictional connection to a database
    email = await conn.execute('select email from users where id=$1', user_id)
    if email is None:
        raise RuntimeError('user not found')
    else:
        return email


async def main():
    email = await get_user_email(123)
    print(email)
    #> testing@example.com
    try:
        await get_user_email(-4)
    except ValidationError as exc:
        print(exc.errors())
        """
        [
            {
                'loc': ('user_id',),
                'msg': 'ensure this value is greater than 0',
                'type': 'value_error.number.not_gt',
                'ctx': {'limit_value': 0},
            },
        ]
        """


asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [20]:
import datetime
from typing import List

from dateutil.parser import parse


order_json = {
    'item_id': '123',
    'created_date': '2002-11-24 12:22',
    'pages_visited': [1, 2, '3'],
    'price': 17.22
}


class Order:
    def __init__( self, item_id: int, created_date: datetime.datetime, pages_visited: List[int], price: float ):
        self.item_id = item_id
        self.created_date = created_date
        self.pages_visited = pages_visited
        self.price = price
        
    def __str__(self):
        return str(self.__str__)

In [21]:
o = Order( **order_json )
print(o)

<bound method Order.__str__ of <__main__.Order object at 0x000001EF627E71C0>>


## Results:
## How to manage [ 1 , 2, '3']

In [26]:

class Order:

    def __init__(self, item_id: int, created_date: datetime.datetime, price: float, pages_visited=None):
        if pages_visited is None:
            pages_visited = []

        try:
            self.item_id = int(item_id)
        except ValueError:
            raise Exception("Invalid item_id, it must be an integer.")

        try:
            self.created_date = parse(created_date)
        except:
            raise Exception("Invalid created_date, it must be an datetime.")

        try:
            self.price = float(price)
        except ValueError:
            raise Exception("Invalid price, it must be an float.")

        try:
            self.pages_visited = [int(p) for p in pages_visited]
        except:
            raise Exception("Invalid page list, it must be iterable and contain only integers.")

    def __str__(self):
        return f'item_id={self.item_id}, created_date={repr(self.created_date)}, ' \
               f'price={self.price}, pages_visited={self.pages_visited}'

    def __eq__(self, other):
        return isinstance(other, Order) and self.__dict__ == other.__dict__

    def __ne__(self, other):
        return isinstance(other, Order) and self.__dict__ == other.__dict__
    

In [27]:
o = Order(**order_json)
print(o)

item_id=123, created_date=datetime.datetime(2002, 11, 24, 12, 22), price=17.22, pages_visited=[1, 2, 3]


## Using pydantic

In [28]:
import datetime
from typing import List, Optional

from dateutil.parser import parse
from pydantic import BaseModel

In [29]:
order_json = {
    'item_id': '123',
    'created_date': '2002-11-24 12:22',
    'pages_visited': [1, 2, '3'],
    'price': 17.22
}

In [30]:

class Order(BaseModel):
    item_id: int
    created_date: Optional[datetime.datetime]
    pages_visited: List[int] = []
    price: float
        
    

In [31]:
o = Order(**order_json)
print(o)

item_id=123 created_date=datetime.datetime(2002, 11, 24, 12, 22) pages_visited=[1, 2, 3] price=17.22
